# Profiling Code Execution with cProfile


In [1]:
import cProfile, pstats, io
from pstats import SortKey
import pandas as pd
import time
from signal_processing_utilities import process_signal

from importlib.util import spec_from_loader, module_from_spec
from importlib.machinery import SourceFileLoader
from scipy.io import wavfile

In [2]:
# Custom import of local file "decode"
spec = spec_from_loader("decode", SourceFileLoader("decode", "../../decode"))
decode = module_from_spec(spec)
spec.loader.exec_module(decode)

# Custom import of local file "encode"
spec = spec_from_loader("encode", SourceFileLoader("encode", "../../encode"))
encode = module_from_spec(spec)
spec.loader.exec_module(encode)

In [3]:
def print_full(data_frame):
    """This function will display all rows of the dataframe.

    Args:
        data_frame (DataFrame): This is a pandas dataframe of
                                arbitrary length.
    """
    pd.set_option("display.max_rows", len(data_frame))
    pd.set_option("display.width", 1000)
    print(data_frame)
    pd.reset_option("display.max_rows")
    pd.reset_option("display.width")

In [4]:
file = "../../data/0ab237b7-fb12-4687-afed-8d1e2070d621.wav"
compressed_file_path = "../../data/0ab237b7-fb12-4687-afed-8d1e2070d621.wav.brainwire"
decompressed_file_path = "../../data/0ab237b7-fb12-4687-afed-8d1e2070d621.wav.copy"

In [16]:
parser = encode.initialize_argument_parser()
args = parser.parse_args([file, compressed_file_path, "-m=u"])
start_time = time.time_ns()
encode.main(args)
stop_time = time.time_ns()
process_signal.print_time_each_function_takes_to_complete_processing(
    start_time, stop_time, executed_line="unique huffman compression"
)
process_signal.print_size_of_file_compression(
    file_path=file, compressed_file_path=compressed_file_path
)


decode_parser = decode.initialize_argument_parser()
decode_args = decode_parser.parse_args([compressed_file_path, decompressed_file_path])
start_time = time.time_ns()
decode.main(decode_args)
stop_time = time.time_ns()
process_signal.print_time_each_function_takes_to_complete_processing(
    start_time, stop_time, executed_line="decode unique huffman compression"
)
process_signal.print_size_of_file_compression(
    file_path=file, compressed_file_path=compressed_file_path
)


Executed Line: unique huffman compression...
Time Δ Nanoseconds: 148734000
Time Δ Microseconds: 148734.0
Time Δ Milliseconds: 148.734
Time Δ Seconds: 0.148734


Original File Size: 197526
Compressed File Size: 74054
Percent of Compression: 62.51%
Percent of Compressed File Size Relative to Required File Size: 7502.938%

Executed Line: decode unique huffman compression...
Time Δ Nanoseconds: 1995818000
Time Δ Microseconds: 1995818.0
Time Δ Milliseconds: 1995.818
Time Δ Seconds: 1.995818


Original File Size: 197526
Compressed File Size: 74054
Percent of Compression: 62.51%
Percent of Compressed File Size Relative to Required File Size: 7502.938%


In [6]:
wavfile.read(decompressed_file_path)

(19531, array([1697, 1889, 1569, ...,  928, 1184, 1504], dtype=int16))

In [7]:
wavfile.read(file)

(19531, array([1697, 1889, 1569, ...,  928, 1184, 1504], dtype=int16))

In [8]:
args = parser.parse_args([file, compressed_file_path, "-m=u"])

In [9]:
pr = cProfile.Profile()
pr.enable()
encode.main(args)
s = io.StringIO()
sortby = SortKey.TIME
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         180679 function calls (180429 primitive calls) in 0.194 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.090    0.090    0.095    0.095 ../../encode:141(create_byte_string)
    98741    0.043    0.000    0.043    0.000 {method 'index' of 'list' objects}
        1    0.019    0.019    0.019    0.019 ../../encode:66(determine_hex_freq)
        1    0.010    0.010    0.053    0.053 ../../encode:489(<listcomp>)
        1    0.009    0.009    0.009    0.009 ../../encode:124(<listcomp>)
        1    0.005    0.005    0.005    0.005 ../../encode:127(<listcomp>)
    72359    0.003    0.000    0.003    0.000 {method 'to_bytes' of 'int' objects}
        1    0.002    0.002    0.002    0.002 {method 'sort' of 'numpy.ndarray' objects}
        2    0.002    0.001    0.002    0.001 {built-in method numpy.asanyarray}
        2    0.002    0.001    0.002    0.001 {built-in method numpy.array}
        2    0.001    0.

In [10]:
s.getvalue()

"         180679 function calls (180429 primitive calls) in 0.194 seconds\n\n   Ordered by: internal time\n\n   ncalls  tottime  percall  cumtime  percall filename:lineno(function)\n        1    0.090    0.090    0.095    0.095 ../../encode:141(create_byte_string)\n    98741    0.043    0.000    0.043    0.000 {method 'index' of 'list' objects}\n        1    0.019    0.019    0.019    0.019 ../../encode:66(determine_hex_freq)\n        1    0.010    0.010    0.053    0.053 ../../encode:489(<listcomp>)\n        1    0.009    0.009    0.009    0.009 ../../encode:124(<listcomp>)\n        1    0.005    0.005    0.005    0.005 ../../encode:127(<listcomp>)\n    72359    0.003    0.000    0.003    0.000 {method 'to_bytes' of 'int' objects}\n        1    0.002    0.002    0.002    0.002 {method 'sort' of 'numpy.ndarray' objects}\n        2    0.002    0.001    0.002    0.001 {built-in method numpy.asanyarray}\n        2    0.002    0.001    0.002    0.001 {built-in method numpy.array}\n        

In [11]:
with cProfile.Profile() as pr:

    encode.main(args)

df = pd.DataFrame(
    pr.getstats(),
    columns=["func", "ncalls", "ccalls", "tottime", "cumtime", "callers"],
)

In [12]:
df[df["cumtime"] == df["cumtime"].max()]

,func,ncalls,ccalls,tottime,cumtime,callers
54,<code object create_byte_string at 0x12ba58e00...,1,0,0.093329,0.088693,[(<method 'tobytes' of 'numpy.ndarray' objects...


In [13]:
df.sort_values("tottime", ascending=False)

,func,ncalls,ccalls,tottime,cumtime,callers
49,"<code object main at 0x10fdfba90, file ""../../...",1,0,1.939390e-01,3.986670e-04,"[(<code object read at 0x14e74fe00, file ""/Vol..."
47,<code object encode_using_amplitude_indices at...,1,0,1.901935e-01,1.704990e-04,[(<code object huffman_encoding at 0x12ba56c00...
54,<code object create_byte_string at 0x12ba58e00...,1,0,9.332933e-02,8.869346e-02,[(<method 'tobytes' of 'numpy.ndarray' objects...
74,"<code object <listcomp> at 0x158654130, file ""...",1,0,5.364554e-02,1.030833e-02,"[(<method 'index' of 'list' objects>, 98741, 0..."
2,<method 'index' of 'list' objects>,98741,0,4.333722e-02,4.333722e-02,None
...,...,...,...,...,...,...
6,<method 'values' of 'dict' objects>,4,0,4.170000e-07,4.170000e-07,None
21,<method '__exit__' of '_thread.RLock' objects>,1,0,3.330000e-07,3.330000e-07,None
11,<built-in method builtins.isinstance>,2,0,2.920000e-07,2.920000e-07,None
35,<code object _unique_dispatcher at 0x105e62730...,1,0,2.090000e-07,2.090000e-07,None


In [14]:
df.iloc(1)

In [15]:
print_full(df.sort_values("tottime", ascending=False))

                                                 func  ncalls  ccalls       tottime       cumtime                                            callers
49  <code object main at 0x10fdfba90, file "../../...       1       0  1.939390e-01  3.986670e-04  [(<code object read at 0x14e74fe00, file "/Vol...
47  <code object encode_using_amplitude_indices at...       1       0  1.901935e-01  1.704990e-04  [(<code object huffman_encoding at 0x12ba56c00...
54  <code object create_byte_string at 0x12ba58e00...       1       0  9.332933e-02  8.869346e-02  [(<method 'tobytes' of 'numpy.ndarray' objects...
74  <code object <listcomp> at 0x158654130, file "...       1       0  5.364554e-02  1.030833e-02  [(<method 'index' of 'list' objects>, 98741, 0...
2                  <method 'index' of 'list' objects>   98741       0  4.333722e-02  4.333722e-02                                               None
53  <code object huffman_encoding at 0x12ba56c00, ...       1       0  3.510575e-02  1.050753e-03  [(<meth